## Rag implementation

In [5]:
print("this is RAG tutorial...")

this is rag tutorial...


## Load steps 1)

In [6]:
from langchain_community.document_loaders import TextLoader

In [7]:
loader = TextLoader('earth.txt')

In [8]:
docs = loader.load()

In [10]:
type(docs)

list

In [11]:
docs

[Document(metadata={'source': 'earth.txt'}, page_content='Earth, the third planet from the Sun, is a remarkable celestial body teeming with life. Unique in our solar system, it is often called the "Blue Planet" due to the abundant water covering about 71% of its surface. Earthâ€™s complex system of land, water, air, and life has allowed for the development of a rich biosphere, making it distinct from other planets.\n\nThe planet is about 4.5 billion years old and has a layered structure, consisting of the crust, mantle, outer core, and inner core. The crust is Earth\'s outermost layer, where we live, and itâ€™s broken into large pieces called tectonic plates. These plates move slowly but are responsible for many geological phenomena like earthquakes, volcanic eruptions, and the formation of mountains. Below the crust lies the mantle, a thick layer of semi-molten rock, which drives plate tectonics through convection currents. At the core, primarily composed of iron and nickel, Earth\'s 

In [17]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")

In [19]:
from langchain_community.document_loaders import WebBaseLoader
import bs4

loader=WebBaseLoader(web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
                     bs_kwargs=dict(parse_only=bs4.SoupStrainer(
                         class_=("post-title","post-content","post-header")

                     )))

text_documents=loader.load()

In [20]:
text_documents

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='\n\n      LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\nBuilding agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview#\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistake

In [21]:
type(text_documents)

list

In [24]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader('encoder-decoder.pdf')
docs = loader.load()

In [25]:
docs

[Document(metadata={'source': 'encoder-decoder.pdf', 'page': 0}, page_content='Understanding How Encoder-Decoder\nArchitectures Attend\nKyle Aitken\nDepartment of Physics\nUniversity of Washington\nSeattle, Washington, USA\nkaitken17@gmail.comVinay V Ramasesh\nGoogle Research, Blueshift Team\nMountain View, California, USA\nYuan Cao\nGoogle Research, Brain Team\nMountain View, California, USANiru Maheswaranathan\nGoogle Research, Brain Team\nMountain View, California, USA\nAbstract\nEncoder-decoder networks with attention have proven to be a powerful way to solve\nmany sequence-to-sequence tasks. In these networks, attention aligns encoder and\ndecoder states and is often used for visualizing network behavior. However, the\nmechanisms used by networks to generate appropriate attention matrices are still\nmysterious. Moreover, how these mechanisms vary depending on the particular\narchitecture used for the encoder and decoder (recurrent, feed-forward, etc.) are also\nnot well understood

## Splitting steps 2)

In [27]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
chunk_docs = splitter.split_documents(docs)
chunk_docs

[Document(metadata={'source': 'encoder-decoder.pdf', 'page': 0}, page_content='Understanding How Encoder-Decoder\nArchitectures Attend\nKyle Aitken\nDepartment of Physics\nUniversity of Washington\nSeattle, Washington, USA\nkaitken17@gmail.comVinay V Ramasesh\nGoogle Research, Blueshift Team\nMountain View, California, USA\nYuan Cao\nGoogle Research, Brain Team\nMountain View, California, USANiru Maheswaranathan\nGoogle Research, Brain Team\nMountain View, California, USA\nAbstract\nEncoder-decoder networks with attention have proven to be a powerful way to solve\nmany sequence-to-sequence tasks. In these networks, attention aligns encoder and\ndecoder states and is often used for visualizing network behavior. However, the\nmechanisms used by networks to generate appropriate attention matrices are still\nmysterious. Moreover, how these mechanisms vary depending on the particular\narchitecture used for the encoder and decoder (recurrent, feed-forward, etc.) are also\nnot well understood

## Embedding technique steps 3)

In [28]:
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings

## FAISS - Facebook AI similarity search

from langchain_community.vectorstores import FAISS
db = FAISS.from_documents(chunk_docs,OpenAIEmbeddings())


C:\Users\ayush\AppData\Local\Temp\ipykernel_17156\85972367.py:7: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  db = FAISS.from_documents(chunk_docs,OpenAIEmbeddings())


In [29]:
db

In [30]:
type(db)

langchain_community.vectorstores.faiss.FAISS

In [33]:
query = "We develop a method for decomposing the hidden states of the network into a sum of components that let us isolate input driven behavior from temporal (or sequence) driven behavior."

In [34]:
retrieve_result = db.similarity_search(query)
print(retrieve_result[0].page_content)

Our Contributions
•We propose a decomposition of hidden state dynamics into separate pieces, one of which explains
the temporal behavior of the network, another of which describes the input behavior. We show
such a decomposition aids in understanding the behavior of networks with attention.
•In the tasks studied, we show the temporal (input) components play a larger role in determining
the attention matrix as the average attention matrix becomes a better (worse) approximation for a
random sample’s attention matrix.
•We discuss the dynamics of architectures with attention and/or recurrence and show how the
input/temporal component behavior differs across said architectures.
•We investigate the detailed temporal and input component dynamics in a synthetic setting to
understand the mechanism behind common sequence-to-sequence structures and how they might
differ in the presence of recurrence.
Related Work As mentioned in the introduction, a common technique to gain some understanding
